In [ ]:
import pandas as pd
import os
from scipy import stats
import numpy as np
from os import listdir
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
csv_path = "C:\\Users\\roderickzzc\\Desktop\\Quant project\\data\\"
stockname="close"
close = pd.read_csv(csv_path + stockname + ".csv", index_col = 0, parse_dates = True)
stockname="open"
op = pd.read_csv(csv_path + stockname + ".csv", index_col = 0, parse_dates = True)
stockname="high"
high = pd.read_csv(csv_path + stockname + ".csv", index_col = 0, parse_dates = True)
stockname="low"
low = pd.read_csv(csv_path + stockname + ".csv", index_col = 0, parse_dates = True)
stockname="TURNOVER_VALUE"
turnover = pd.read_csv(csv_path + stockname + ".csv", index_col = 0, parse_dates = True)
stockname="vol"
vol = pd.read_csv(csv_path + stockname + ".csv", index_col = 0, parse_dates = True)

vol=vol.loc['2007-01-04':'2018-08-01']/1000000
turnover=turnover.loc['2007-01-04':'2018-08-01']/1000000
turnover=turnover[vol.columns]
high=high.loc['2007-01-04':'2018-08-01']
low=low.loc['2007-01-04':'2018-08-01']
op=op.loc['2007-01-04':'2018-08-01']
close=close.loc['2007-01-04':'2018-08-01']

forward_day = 5
stockforward = close.shift(periods = -forward_day, freq = None, axis = 0)
stockforwardreturn = (stockforward - close) / close

In [ ]:
stockforwardreturn
#stockforwardreturn=stockforwardreturn.set_index(close.index).set_axis(close.columns)
stockforwardreturn.to_csv(os.path.join(csv_path, '5DForward.csv'))

In [ ]:
csv_path = "C:\\Users\\roderickzzc\\Desktop\\Data\\Backtest\\"
indexclose = "IndexClosePrice"
index = pd.read_csv(csv_path + indexclose + ".csv", index_col = 0, parse_dates = True)
indexforward = index.shift(periods = -forward_day, freq = None, axis = 0)
indexforwardreturn = (indexforward - index) / index
indexforwardreturn=indexforwardreturn.iloc[:-6].set_index(close.index)
indexforwardreturn.to_csv(os.path.join('C:\\Users\\roderickzzc\\Desktop\\Quant project\\data\\', 'mkt_5DForward.csv'))

In [ ]:
def _rank(data):
    return pd.DataFrame.rank(close,axis=1)

def _rolling_rank(data):
    value = stats.rankdata(data,axis=0)[-1]
    return value

def _cube(data):
    return np.square(data)*data

def _square(data):
    return np.square(data)

def _delta(data):
    delta=np.diff(data,axis=0)
    delta=np.insert(delta, 0, 0, axis=0)
    return pd.DataFrame(delta).set_index(data.index).set_axis(data.columns, axis=1,inplace=False)

def _delay(data,n):
    period=n
    value = data.shift(period)
    value = np.nan_to_num(value)
    return pd.DataFrame(value).set_index(data.index).set_axis(data.columns, axis=1,inplace=False)
        
def _corr(data1,data2,n):
    return data1.rolling(n).corr(data2)

def _cov(data1,data2,n):
    return data1.rolling(n).cov(data2)
    
def _ts_sum(data,n):
    dataa=data.rolling(n).sum()
    dataa=np.nan_to_num(dataa)
    return pd.DataFrame(dataa).set_index(data.index).set_axis(data.columns, axis=1,inplace=False)
        
def _sma(data,n):
    dataa=data.rolling(n).mean()
    dataa=np.nan_to_num(dataa)
    return pd.DataFrame(dataa).set_index(data.index).set_axis(data.columns, axis=1,inplace=False)

def _stddev(data,n): 
    dataa=data.rolling(n).std()
    dataa=np.nan_to_num(dataa)
    return pd.DataFrame(dataa).set_index(data.index).set_axis(data.columns, axis=1,inplace=False)

def _ts_rank(data,n):
    dataa=data.rolling(n).apply(_rolling_rank)
    dataa=np.nan_to_num(dataa)
    return pd.DataFrame(dataa).set_index(data.index).set_axis(data.columns, axis=1,inplace=False)
    
def _ts_argmin(data,n):
    dataa=data.rolling(n).apply(np.argmin)
    dataa=np.nan_to_num(dataa)
    return pd.DataFrame(dataa).set_index(data.index).set_axis(data.columns, axis=1,inplace=False)

def _ts_argmax(data,n):
    dataa=data.rolling(n).apply(np.argmax)
    dataa=np.nan_to_num(dataa)
    return pd.DataFrame(dataa).set_index(data.index).set_axis(data.columns, axis=1,inplace=False)
        
def _ts_min(data,n):
    dataa=data.rolling(n).min()
    dataa=np.nan_to_num(dataa)
    return pd.DataFrame(dataa).set_index(data.index).set_axis(data.columns, axis=1,inplace=False)
        
def _ts_max(data,n):
    dataa=data.rolling(n).min()
    dataa=np.nan_to_num(dataa)
    return pd.DataFrame(dataa).set_index(data.index).set_axis(data.columns, axis=1,inplace=False)
        
def _ts_argmaxmin(data,n):
    return _ts_argmax(data,n) - _ts_argmin(data,n)

def protectedDiv(data1,data2):
    df=data1/data2
    return df

def linear_weight(d):
    w = list([n/sum(range(1,d+1)) for n in range(1,d+1)])
    def g(x):
        return sum(w*x) 
    return g

def linear_decay(data,n):
    return data.rolling(window=n).apply(linear_weight(n))

def exponential_average(data):
    return data.ewm(com=0.5).mean()

def iden(data):
    return data

In [ ]:
%cd C:\\Users\\roderickzzc\\Desktop\\Data\\Backtest

In [ ]:
save_result_path =  'C:\\Users\\roderickzzc\\Desktop\\Data\\New_factors\\'

In [ ]:
alpha1=vol+linear_decay(vol, 7)
alpha1.to_csv(os.path.join(save_result_path, 'alpha1.csv'))

In [ ]:
alpha2=linear_decay(op, 2)/(high-low)
alpha2.replace([np.inf, -np.inf], np.nan,inplace=True)
alpha2.to_csv(os.path.join(save_result_path, 'alpha2.csv'))

In [ ]:
alpha3=vol/_ts_rank(op, 20)
alpha3.replace([np.inf, -np.inf], np.nan,inplace=True)
alpha3.to_csv(os.path.join(save_result_path, 'alpha3.csv'))

In [ ]:
alpha4=_cube(close)-_corr(close, turnover, 5)

In [ ]:
alpha4.replace([np.inf, -np.inf], np.nan,inplace=True)
alpha4.to_csv(os.path.join(save_result_path, 'alpha4.csv'))

In [ ]:
alpha5=linear_decay(vol, 14) 
alpha5.to_csv(os.path.join(save_result_path, 'alpha5.csv'))

In [ ]:
alpha6=(vol/turnover)/_ts_sum(high, 2)
alpha6.replace([np.inf, -np.inf], np.nan,inplace=True)
alpha6.to_csv(os.path.join(save_result_path, 'alpha6.csv'))

In [ ]:
alpha7=vol+_corr(close, op, 6)
alpha7.replace([np.inf, -np.inf], np.nan,inplace=True)

In [ ]:
alpha8=vol
alpha8.to_csv(os.path.join(save_result_path, 'alpha8.csv'))

In [ ]:
alpha9=close
alpha9.replace([np.inf, -np.inf], np.nan,inplace=True)
alpha9.to_csv(os.path.join(save_result_path, 'alpha9.csv'))

In [ ]:
%run helper.py
%run backtestlite.py

working_directory = 'C:\\Users\\roderickzzc\\Desktop\\Quant project\\data\\'
factor_path_all = 'C:\\Users\\roderickzzc\\Desktop\\Data\\New_factors\\'

direction = "Descending"

#start = '2007-01-04' 
#end = '2017-01-16'
start = '2017-01-17' 
end = '2018-08-01'
quantile = 10
cycle = 5 # Adjustment Cycle (trading frequency), delay for holidays. 
win = 48 # 48 weeks in a year
year = 2017 # Beginning year of result display (graphs).

#Import stock 5days forward return.
fwdrtn = pd.read_csv(os.path.join(working_directory, "5DForward.csv"), index_col = 0, parse_dates = True).loc[start:end][::cycle]
#Import market 5days forward return.
mkt_index = pd.read_csv(os.path.join(working_directory, "mkt_5DForward.csv"), index_col = 0, parse_dates = True).loc[str(start):str(end),"SH000300"][::cycle]
status = pd.DataFrame(1, index = fwdrtn.index, columns = fwdrtn.columns)
statuslimit = pd.DataFrame(1, index = fwdrtn.index, columns = fwdrtn.columns)

# Revise the factor name (name of .csv file)
factor = "alpha7"

Type = "LS" # this stand for long-short strategy. 


def singlefactor(factor,factor_path_all,Type,direction,fwdrtn,mkt_index,cycle, start, end, quantile,win,year, statuslimit):
    factor_path = os.path.join(factor_path_all, factor + '.csv')

    Type1 = "LS"
    B = Backtestlite(factor_path, direction, fwdrtn, mkt_index, factor_path_all, cycle, start, end, quantile, statuslimit)
    B.run(Type1)

    # Drawing
    pic = DrawPicture(factor,direction, B.data,  B.fwdrtn, B.data.index , mkt_index,  B.rtn_data, B.coverage, B.turnover, factor_path_all, Type1)    
    pic.main(win,year,B.rtn_data.columns)
    
# Calculate performances and do the plot
singlefactor(factor,factor_path_all,Type, direction,fwdrtn,mkt_index,cycle, start, end, quantile,win,year, statuslimit)
plt.show()